In [69]:
import numpy as np
import matplotlib.pyplot as plt
%load_ext watermark
%watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
2022-02-17T17:37:10+01:00

CPython 3.6.7
IPython 6.4.0

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-167-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


In [81]:
# %matplotlib inline
%matplotlib qt

# 1. Trapped ions physics
## Motion equation
In 1D the motion equation of a trapped ion $i$, in an RF field, with friction and Coulomb interaction is as follows :

$m\ddot y_i = \frac{2q_eU_{RF}}{r_0^2}\cos\Omega t ~y_i + q_e^2k_C\sum_{j=1}^N \frac{y_i-y_j}{d_{ij}^3} - \gamma\dot y_i$

Numerically solve this equation is equivalent to obtain the set of variables $y_i$, $v_i=\frac{\mathrm{d}y_i}{\mathrm{d}t}$ and $a_i = \frac{\mathrm{d^2}y_i}{\mathrm{d}t^2}$ that fulfill this equation. Several methods can be used. In our Fortran code, when there are no friction velocity-Verlet method (VV) is used, but when friction is implemented another method must be used. Blümel *et al* use Runge-Kutta fourth order method (RK4). This document deals with both VV and RK4.

For RK4 it is convenient to write the second order differential equation as a system of two first order differential equations

$\dot y_i = v_i \\ \dot v_i = a_i = \frac{2q_eU_{RF}}{r_0^2}\cos\Omega t ~y_i + q_e^2k_C\sum_{j=1}^N \frac{y_i-y_j}{d_{ij}^3} - \gamma v_i$

It may seems that this decomposition is artificial, expecially because there is no analytical expression for the velocity $v_i$. Nevertheless this is no problem because the actual value for the velocity $v_i$ is know at the beginning of each step of RK4 algorithm.

## Trapping parameters
$q_u = \frac{4q_e}{mr_0^2\Omega^2}U_{RF}$, $a_z = \frac{8\kappa q_e}{md_0^2\Omega^2}U_{DC}$. For a given $q_u$, $U_{RF} = \frac{q_umr_0^2\Omega^2}{4q_e}$.

The cloud is spherical for $\omega_z = \omega_x \leftrightarrow a_z = \frac{q_u^2}{2}$, thus $U_{DC} = \frac{d_0^2q_e}{r_0^4m\kappa\Omega^2}U_{RF}^2$

# 2. Simulation of a trapped ion

Done with ForTran ```Runge_Kutta_Nions.F90``` as in this [Stack exchange](https://math.stackexchange.com/questions/721076/help-with-using-the-runge-kutta-4th-order-method-on-a-system-of-2-first-order-od) (but I have seen this elsewhere)

In [72]:
# definition of physical variables
C_e = 1.602e-19
ke  = 8.987551787e9
m_Ca = 40.078*1.66054e-27 # mass of trapped ion
r0 = 2.865e-3/1.14511  # trap internal radius 2.5e-3
d0 = 4e-3/2
Omega = 2*np.pi*2e6  # RF field angular frequency
Urf = 40       # RF voltage amplitude

n_dt = 100
dt_rk4 = 2*np.pi/(n_dt*Omega)

In [73]:
kappa = (2*np.pi*90806.9982303)**2*m_Ca*d0**2/(2*C_e)
print(kappa)
q_u = 0.3
a_u = 0
a_z = 8*kappa*C_e/(m_Ca*d0**2*Omega**2)
print('q_u , a_z =>',q_u,',',a_z)
Urf = q_u*m_Ca*r0**2*Omega**2/(4*C_e)
Udc = Urf**2 *d0**2*C_e/(r0**4*m_Ca*Omega**2*kappa)
print(Urf,Udc)

w_r = 0.5*Omega*np.sqrt(0.5*q_u**2+a_u-0.5*a_z)
print(w_r)

0.27047133360403336
q_u , a_z => 0.3 , 0.008245910927597709
30.798438655045498 5.457250314139631
1270338.9447828738


In [74]:
# In Tarnas, Blumel etcetc
# dimensionless equation
# here is the conversion factor to go
# from gamma in dimensionless to gamma in usi
gamma_blu = 1e-2
tc = 2/Omega
gamma_usi = gamma_blu * m_Ca/tc
print(gamma_usi)
print('nt =',10/gamma_blu) # nt number of rf cycles for initialisation

4.1815303268069835e-21
nt = 1000.0


# Loading data from Fortran version
```gfortran -o3 .\Runge_Kutta_ions.f90 -o a.exe; .\a.exe```

In [166]:
str_N = '0016'
str_gam ='0.42D-20'

In [167]:
# Loading dynamics of a single ion in the simu                xyz_1ion_N1024_F0.10D-19_RK4BlumelStyle
fileloc = '/home/adrian/RemoteFS/Rivendel/Simulations/20220208/xyz_1ion_N' \
          + str_N + '_F' \
          + str_gam + '_RK4BlumelStyle.dat'
with open(fileloc) as f:
    all_data = np.loadtxt(f,unpack=True) 
    t_For = all_data[0]
    r_For = all_data[1:4]
    v_For = all_data[4:7]
    a_For = all_data[7:10]

In [168]:
# plot the dynamics, r,v,a of the recorded ion
plt.figure('First ion dynamics',clear='True')
ax = plt.subplot(111)

ax.plot(t_For/(n_dt*dt_rk4),np.array(r_For[0])/r0,ls=':',label='x RK4 Fortran')
# ax.plot(t_For/(n_dt*dt_rk4),np.array(r_For[1])/r0,ls=':',label='y RK4 Fortran')
# ax.plot(t_For/(n_dt*dt_rk4),np.array(r_For[2])/r0,ls=':',label='z RK4 Fortran')

# ax.plot(t_For/(n_dt*dt_rk4),v_For[0],ls=':',marker='+',label='v_x RK4 Fortran')
# ax.plot(t_For/(n_dt*dt_rk4),a_For[0],ls=':',label='a_x RK4 Fortran')

ax.set_xlabel('time [RF period]')
ax.set_ylabel('x [$r_0$]')
# ax.set_ylim(-5,1e5)
ax.grid()
ax.legend()
plt.tight_layout()

In [169]:
# Loading position of ions at given time                      
# endfly is at end of simu
# endinit is at end init
fileloc = '/home/adrian/RemoteFS/Rivendel/Simulations/20220208/r_endfly_N' \
          + str_N + '_F' \
          + str_gam + '_RK4BlumelStyle.dat'
with open(fileloc) as f:
    all_data = np.loadtxt(f,unpack=True) 
    x_photo = all_data[0,:]
    y_photo = all_data[1,:]
    z_photo = all_data[2,:]

In [170]:
# plot position of ions at that given time
plt.figure('Ion positions',clear='True')
ax = plt.subplot(121)
ax.scatter(x_photo/r0,y_photo/r0,marker='o',facecolor='',edgecolor='b')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.grid()

ax = plt.subplot(122)
ax.scatter(z_photo/d0,x_photo/r0,facecolor='',edgecolor='b')
ax.set_xlabel('z')
ax.set_ylabel('x')
# ax.set_ylim(-5,1e5)
ax.grid()

plt.tight_layout()

In [171]:
# Loading temperature of ions
fileloc = '/home/adrian/RemoteFS/Rivendel/Simulations/20220208/T_N' \
          + str_N + '_F' \
          + str_gam + '_RK4BlumelStyle.dat'
with open(fileloc) as f:
    all_data = np.loadtxt(f,unpack=True)
    t_temp = all_data[0]
    T_CM   = np.mean(all_data[1:4],axis=0)
    T_aux  = np.mean(all_data[4:7],axis=0)
    T_x = all_data[4]

In [172]:
# plot temperature
plt.figure('Ions temperature',clear='True')
ax = plt.subplot(111)

ax.plot(t_temp/(n_dt*dt_rk4),T_aux,ls=':',marker='+')
# ax.plot(t_temp/(n_dt*dt_rk4),T_x,ls=':',marker='+')
# ax.plot(t_For/(n_dt*dt_rk4),a_For[0],ls=':',label='a_x RK4 Fortran')

ax.set_xlabel('time [RF period]')
ax.set_ylabel('T [K]')
# ax.set_ylim(-5,1e5)
ax.grid()
ax.legend()

No handles with labels found to put in legend.


In [173]:
plt.figure('time fro temperature',clear='True')
ax = plt.subplot(111)

ax.plot(t_temp[:len(t_temp)//2],ls=':',marker='+')
# ax.plot(t_For,ls=':',label='a_x RK4 Fortran')